In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import seaborn as sbn


df=pd.read_csv('trainOr.csv')
df_test=pd.read_csv('testOr.csv')
df_test

,Unnamed: 0,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Relatives,Alone,Title
0,0,3,1,34.5,0,0,7.8292,1,0,0.0,2
1,1,3,0,47.0,1,0,7.0000,2,1,1.0,3
2,2,2,1,62.0,0,0,9.6875,1,0,0.0,2
3,3,3,1,27.0,0,0,8.6625,2,0,0.0,2
4,4,3,0,22.0,1,1,12.2875,2,2,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...
413,413,3,1,32.0,0,0,8.0500,2,0,0.0,2
414,414,1,0,39.0,0,0,108.9000,0,0,0.0,4
415,415,3,1,38.5,0,0,7.2500,2,0,0.0,2
416,416,3,1,32.0,0,0,8.0500,2,0,0.0,2


In [3]:
X_train = df.drop("Survived", axis=1)
y_train = df.Survived

X_test = df_test.copy()

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix


In [6]:
classifiers = {
    "NB": GaussianNB(),
    "SGD": SGDClassifier(),
    "KNN": KNeighborsClassifier(), 
    "LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2),
    "SVM": SVC(),
    "MLP": MLPClassifier(max_iter=1000),
    "XGB": XGBClassifier(),
    "LGBM": LGBMClassifier()
}

results = pd.DataFrame(columns=["Classifier", "Avg_Accuracy", "Avg_F1_Score"])
for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(
        model, X_train_scaled, y_train, cv=10,
        scoring=(['accuracy', 'f1'])
    )

    results = results.append({
        "Classifier": name,
        "Avg_Accuracy": cv_results['test_accuracy'].mean(),
        "Avg_F1_Score": cv_results['test_f1'].mean()
    }, ignore_index=True)
    
results["Avg_Overall"] = (results["Avg_Accuracy"] + results["Avg_F1_Score"]) / 2
results = results.sort_values("Avg_Overall", ascending=False)
results

C:\Users\mehmu\AppData\Local\Temp\ipykernel_23768\1442918491.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_23768\1442918491.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_23768\1442918491.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_23768\1442918491.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
C:\Users\mehmu\AppData\Local\Temp\ipykernel_23768\1442918491.py:22: FutureWarnin

,Classifier,Avg_Accuracy,Avg_F1_Score,Avg_Overall
6,SVM,0.829413,0.763450,0.796431
5,RF,0.824906,0.753168,0.789037
7,MLP,0.821561,0.750831,0.786196
0,NB,0.799126,0.746141,0.772633
2,KNN,0.804707,0.733640,0.769173
3,LR,0.803596,0.730779,0.767187
9,LGBM,0.791273,0.733673,0.762473
8,XGB,0.771036,0.696760,0.733898
1,SGD,0.752060,0.681010,0.716535
4,DT,0.744120,0.679039,0.711579


In [25]:
svm = KNeighborsClassifier()
params = {
    "weights": ('uniform', 'distance'),
    'leaf_size':(5,6,7,8,9,10,15,17,20,23),
    'n_neighbors':(6,7,8,9,10,11,12,14,16,18,20,22)
}
clf = GridSearchCV(svm, params, cv=10)
clf.fit(X_train_scaled, y_train)
print("Best hyperparameter:", clf.best_params_)

Best hyperparameter: {'leaf_size': 5, 'n_neighbors': 10, 'weights': 'uniform'}


In [7]:
clf = SVC(C=10,tol=0.1)
clf.fit(X_train_scaled, y_train)
#Score: 0.76555

SVC(C=10, tol=0.1)

In [12]:
clf=LogisticRegression()
clf.fit(X_train_scaled, y_train)
#Score: 0.77033

LogisticRegression()

In [34]:
clf=KNeighborsClassifier(algorithm='auto', leaf_size=26, metric='minkowski',metric_params=None, n_jobs=1, n_neighbors=6, p=2,weights='uniform')
clf.fit(X_train_scaled, y_train)
#Score 0.75

KNeighborsClassifier(leaf_size=26, n_jobs=1, n_neighbors=6)

In [38]:
clf=XGBClassifier()
clf.fit(X_train_scaled, y_train)
#core: 0.77751

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [39]:
y_pred = clf.predict(X_test_scaled)


In [40]:
submission = pd.read_csv("gender_submission.csv")
submission["Survived"] = y_pred
submission.to_csv('submission.csv', index=False)

In [41]:
!kaggle competitions submit -c titanic -f submission.csv -m ""

Successfully submitted to Titanic - Machine Learning from Disaster



  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|██████████| 3.18k/3.18k [00:01<00:00, 2.36kB/s]
